# First Attempt at Kaggle Sample Problems
### https://www.kaggle.com/c/titanic/overview

# Attempt 1: 0.73684 (Empty Values Replace With Mean)

In [1]:
### Imports
# ML
import tensorflow
import keras
import scipy
from sklearn.svm import LinearSVC

# Data
import numpy as np
import pandas as pd
import csv

import math
import re

Using TensorFlow backend.


In [34]:
# Import Data
trainData = pd.read_csv('train.csv', index_col='PassengerId')
trainData.insert(1, "dataType", 0)

testData = pd.read_csv('test.csv', index_col='PassengerId')
testData.insert(0, "Survived", np.NaN)
testData.insert(1, "dataType", 2)

"Got Data"

,Survived,dataType,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0.0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1.0,0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,1.0,0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1.0,0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,0.0,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
1304,NaN,2,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
1305,NaN,2,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
1306,NaN,2,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
1307,NaN,2,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [73]:
# Build Data
dataSet = pd.concat([trainData, testData], join='outer', ignore_index=False)

# Replace Null Values
dataSet = dataSet.fillna({ "Cabin": "NA", "Embarked": "NA", "Age": 29, "Fare": 33 })

#Remove Digits from Cabins
dataSet['Cabin'] = dataSet['Cabin'].apply(lambda x : re.sub(r"\d+", "", x)[0])

# Adjust Genders to Scalar
def adjustSex(x):
    if (x == "male"):
        return 1
    
    if (x == "female"):
        return -1
    
    return x

dataSet['Sex'] = dataSet['Sex'].apply(adjustSex)

# Class
dataSet.insert(dataSet.columns.get_loc('Pclass'), "ThirdClass", dataSet['Pclass'] == 3)
dataSet.insert(dataSet.columns.get_loc('Pclass'), "SecondClass", dataSet['Pclass'] == 2)
dataSet.insert(dataSet.columns.get_loc('Pclass'), "FirstClass", dataSet['Pclass'] == 1)

dataSet = dataSet.drop(columns="Pclass")

# Cabin
for x in dataSet["Cabin"].value_counts().keys().tolist():
    dataSet.insert(dataSet.columns.get_loc('Cabin'), "Cabin" + x, dataSet['Cabin'] == x)

dataSet = dataSet.drop(columns="Cabin")

# Embarked
for x in dataSet["Embarked"].value_counts().keys().tolist():
    dataSet.insert(dataSet.columns.get_loc('Embarked'), "Port" + x, dataSet["Embarked"] == x)

dataSet = dataSet.drop(columns="Embarked")

# Swap False for 0.0 and True for 1.0
dataSet = dataSet.replace(True, 1.0)

droppedColumns = ["Survived", 'dataType', 'Name', 'Ticket']

trainRaw = dataSet.loc[dataSet['dataType'] == 0]
trainX = trainRaw.drop(droppedColumns, axis=1)
trainY = trainRaw["Survived"]

outputRaw = dataSet.loc[dataSet["dataType"] == 2]
outputX = outputRaw.drop(droppedColumns, axis=1)

dataSet

,Survived,dataType,ThirdClass,SecondClass,FirstClass,Name,Sex,Age,SibSp,Parch,...,CabinD,CabinE,CabinA,CabinF,CabinG,CabinT,PortS,PortC,PortQ,PortNA
PassengerId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0,1.0,0.0,0.0,"Braund, Mr. Owen Harris",1.0,22.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.0,0,0.0,0.0,1.0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",-1.0,38.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1.0,0,1.0,0.0,0.0,"Heikkinen, Miss. Laina",-1.0,26.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1.0,0,0.0,0.0,1.0,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",-1.0,35.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5,0.0,0,1.0,0.0,0.0,"Allen, Mr. William Henry",1.0,35.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,NaN,2,1.0,0.0,0.0,"Spector, Mr. Woolf",1.0,29.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1306,NaN,2,0.0,0.0,1.0,"Oliva y Ocana, Dona. Fermina",-1.0,39.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1307,NaN,2,1.0,0.0,0.0,"Saether, Mr. Simon Sivertsen",1.0,38.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [74]:
trainX

,ThirdClass,SecondClass,FirstClass,Sex,Age,SibSp,Parch,Fare,CabinN,CabinC,...,CabinD,CabinE,CabinA,CabinF,CabinG,CabinT,PortS,PortC,PortQ,PortNA
PassengerId,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,0.0,1.0,22.0,1.0,0.0,7.2500,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,1.0,-1.0,38.0,1.0,0.0,71.2833,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1.0,0.0,0.0,-1.0,26.0,0.0,0.0,7.9250,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,1.0,-1.0,35.0,1.0,0.0,53.1000,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5,1.0,0.0,0.0,1.0,35.0,0.0,0.0,8.0500,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887,0.0,1.0,0.0,1.0,27.0,0.0,0.0,13.0000,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
888,0.0,0.0,1.0,-1.0,19.0,0.0,0.0,30.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
889,1.0,0.0,0.0,-1.0,29.0,1.0,2.0,23.4500,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [65]:
clf = LinearSVC(random_state=0, tol=1e-4, max_iter=10000000)
clf.fit(trainX.to_numpy(), trainY.to_numpy())

print(clf.coef_)

print(clf.intercept_)

[[-0.13098927  0.23471939  0.30394839 -0.5142875  -0.01371153 -0.11105035
  -0.04397449  0.00125946 -0.16246901 -0.03245769  0.11713982  0.33047844
   0.51528083  0.18296204  0.22891467 -0.39183543 -0.38033516 -0.05452849
   0.1061197   0.08521793  0.27086937]]
[0.40767851]


In [79]:
predictions = clf.predict(outputX)

outputX.insert(0, "Survived", predictions)

In [87]:
outputX.loc[:, "Survived"].astype(int).to_csv("export.csv", header=True)

Int64Index([ 892,  893,  894,  895,  896,  897,  898,  899,  900,  901,
            ...
            1300, 1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309],
           dtype='int64', name='PassengerId', length=418)